In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Mostrar versiones
print("PyTorch:", torch.__version__)

# Fijar semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


In [ ]:
# Ver qué hay en /kaggle/input
print("Football-Tackes")
!ls /kaggle/input


In [ ]:
DATASET_ROOT = "/kaggle/input/football-tackles/var400/VAR"  # o var_200/VAR si quieres imágenes más pequeñas

print("DATASET_ROOT:", DATASET_ROOT)
!ls "$DATASET_ROOT"



In [ ]:
full_dataset = datasets.ImageFolder(DATASET_ROOT)
class_names = full_dataset.classes
num_classes = len(class_names)

print("Clases:", class_names)
print("Total de imágenes:", len(full_dataset))


In [ ]:
IMG_SIZE = 224  # tamaño típico para ResNet

data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]),
    "val": transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]),
}

# Dataset base solo para leer clases y tamaño
full_dataset = datasets.ImageFolder(DATASET_ROOT)
class_names = full_dataset.classes
num_classes = len(class_names)

print("Clases:", class_names)
print("Total de imágenes:", len(full_dataset))

# indices aleatorios para train / val (80% / 20%)
indices = torch.randperm(len(full_dataset)).tolist()
train_size = int(0.8 * len(full_dataset))
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Dataset con transform de train
train_dataset = Subset(
    datasets.ImageFolder(DATASET_ROOT, transform=data_transforms["train"]),
    train_indices
)

# Dataset con transform de validación
val_dataset = Subset(
    datasets.ImageFolder(DATASET_ROOT, transform=data_transforms["val"]),
    val_indices
)

print("Tamaño train:", len(train_dataset))
print("Tamaño val:", len(val_dataset))


In [ ]:
BATCH_SIZE = 32  # si estás sin GPU y se cuelga, bájalo a 16 o 8

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
# Cargar ResNet18 con pesos de ImageNet
weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)

# Cambiar la última capa para 2 clases (Clean_Tackles, Fouls)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print(model)


In [ ]:
NUM_EPOCHS = 12  # con GPU puedes subir a 15–20, con CPU déjalo en 8–12

history = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": [],
}

best_acc = 0.0
best_state_dict = None

for epoch in range(NUM_EPOCHS):
    # --- TRAIN ---
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += (preds == labels).sum().item()
        total_train += labels.size(0)

    epoch_train_loss = running_loss / total_train
    epoch_train_acc = running_corrects / total_train

    # --- VALIDATION ---
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += (preds == labels).sum().item()
            total_val += labels.size(0)

    epoch_val_loss = val_loss / total_val
    epoch_val_acc = val_corrects / total_val

    history["train_loss"].append(epoch_train_loss)
    history["val_loss"].append(epoch_val_loss)
    history["train_acc"].append(epoch_train_acc)
    history["val_acc"].append(epoch_val_acc)

    if epoch_val_acc > best_acc:
        best_acc = epoch_val_acc
        best_state_dict = model.state_dict()

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] "
          f"Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}")

# Guardar mejores pesos
os.makedirs("weights", exist_ok=True)
torch.save(best_state_dict, "weights/model_best.pth")
print(f"Mejor accuracy en validación: {best_acc:.4f}")


In [ ]:
epochs = range(1, NUM_EPOCHS + 1)

plt.figure()
plt.plot(epochs, history["train_loss"], label="Train Loss")
plt.plot(epochs, history["val_loss"], label="Val Loss")
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Evolución de la pérdida")
plt.grid(True)
plt.legend()
plt.show()

plt.figure()
plt.plot(epochs, history["train_acc"], label="Train Acc")
plt.plot(epochs, history["val_acc"], label="Val Acc")
plt.xlabel("Época")
plt.ylabel("Exactitud")
plt.title("Evolución de la exactitud")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Cargar los mejores pesos
model.load_state_dict(torch.load("weights/model_best.pth", map_location=device))
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap="Blues", values_format="d")
plt.title("Matriz de confusión (validación)")
plt.show()

print("Reporte de clasificación:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))


In [ ]:
import torchvision.utils as vutils

# Tomar un batch de validación
inputs, labels = next(iter(val_loader))
inputs = inputs.to(device)
labels = labels.to(device)

model.eval()
with torch.no_grad():
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

# Pasar algunas imágenes a CPU para mostrarlas
inputs_cpu = inputs.cpu()
labels_cpu = labels.cpu()
preds_cpu = preds.cpu()

# Des-normalizar para ver bien
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

def denorm(img):
    return img * std + mean

fig = plt.figure(figsize=(12, 6))
for i in range(8):  # mostrar 8 ejemplos
    ax = fig.add_subplot(2, 4, i+1)
    img = denorm(inputs_cpu[i]).clamp(0, 1)
    npimg = img.permute(1, 2, 0).numpy()
    ax.imshow(npimg)
    ax.axis("off")
    ax.set_title(f"GT: {class_names[labels_cpu[i]]}\nPred: {class_names[preds_cpu[i]]}")
plt.tight_layout()
plt.show()


-****EXPORTACION A TORCHSCRIPT

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Usaremos CPU para exportación/comparación
device_export = torch.device("cpu")
print("Device para exportación/comparación:", device_export)

print("Clases:", class_names, "  -> num_classes =", num_classes)

# Crear la misma arquitectura que entrenaste
model_orig = models.resnet18(weights=None)
num_ftrs = model_orig.fc.in_features
model_orig.fc = nn.Linear(num_ftrs, num_classes)

# Cargar los pesos entrenados y dejar el modelo en CPU
state_dict = torch.load("weights/model_best.pth", map_location=device_export)
model_orig.load_state_dict(state_dict)
model_orig = model_orig.to(device_export)
model_orig.eval()

print("Modelo original cargado en CPU y listo.")



In [ ]:
import os

example_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device_export)

# Exportamos con trace
ts_model = torch.jit.trace(model_orig, example_input)
ts_model = ts_model.to(device_export)
ts_model.eval()

os.makedirs("weights", exist_ok=True)
ts_path = "weights/model_best_torchscript.pt"
ts_model.save(ts_path)

print(f"Modelo exportado a TorchScript en: {ts_path}")



In [ ]:
import time

# Tomamos hasta 256 imágenes de validación para la comparación
inputs_list = []
labels_list = []

max_images = 256

for batch_inputs, batch_labels in val_loader:
    inputs_list.append(batch_inputs)
    labels_list.append(batch_labels)
    if sum(len(b) for b in labels_list) >= max_images:
        break

inputs_all = torch.cat(inputs_list, dim=0)[:max_images]
labels_all = torch.cat(labels_list, dim=0)[:max_images]

print("Número de imágenes usadas para la comparación:", len(labels_all))


In [ ]:
inputs_cpu = inputs_all.to(device_export)
labels_cpu = labels_all.to(device_export)

start = time.time()
with torch.no_grad():
    outputs_orig = model_orig(inputs_cpu)
    _, preds_orig = torch.max(outputs_orig, 1)
end = time.time()

time_orig = end - start
acc_orig = (preds_orig == labels_cpu).float().mean().item()

print(f"Modelo original (PyTorch) -> Accuracy: {acc_orig:.4f}, "
      f"tiempo total: {time_orig:.4f} s, "
      f"tiempo por imagen: {time_orig/len(labels_cpu):.6f} s")


In [ ]:
# Cargar modelo TorchScript (por si lo usas en otro entorno después)
ts_model = torch.jit.load("weights/model_best_torchscript.pt", map_location=device_export)
ts_model.eval()

start = time.time()
with torch.no_grad():
    outputs_ts = ts_model(inputs_cpu)
    _, preds_ts = torch.max(outputs_ts, 1)
end = time.time()

time_ts = end - start
acc_ts = (preds_ts == labels_cpu).float().mean().item()

print(f"Modelo TorchScript -> Accuracy: {acc_ts:.4f}, "
      f"tiempo total: {time_ts:.4f} s, "
      f"tiempo por imagen: {time_ts/len(labels_cpu):.6f} s")


In [ ]:
coinciden = (preds_orig == preds_ts).float().mean().item()
print(f"Porcentaje de imágenes donde ambos modelos predicen lo mismo: {coinciden*100:.2f}%")


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device_export = torch.device("cpu")
print("Device export:", device_export)

print("Clases:", class_names, "  -> num_classes =", num_classes)

# 1. Recrear ResNet18 y cargar pesos
model_orig = models.resnet18(weights=None)
num_ftrs = model_orig.fc.in_features
model_orig.fc = nn.Linear(num_ftrs, num_classes)

state_dict = torch.load("weights/model_best.pth", map_location=device_export)
model_orig.load_state_dict(state_dict)
model_orig = model_orig.to(device_export)
model_orig.eval()

# 2. Exportar a TorchScript con un NOMBRE NUEVO
example_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device_export)
ts_model = torch.jit.trace(model_orig, example_input)

ts_path = "weights/model_futbol_ts.pt"   # <<< nombre nuevo
ts_model.save(ts_path)

print("✅ TorchScript guardado en:", ts_path)


In [ ]:
test_ts = torch.jit.load("weights/model_futbol_ts.pt", map_location="cpu")
print("✅ TorchScript se carga bien en Kaggle:", type(test_ts))
